In [52]:
library(h2o)

In [53]:
h2o.init()

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 hours 30 minutes 
    H2O cluster timezone:       America/New_York 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.36.0.3 
    H2O cluster version age:    9 days  
    H2O cluster name:           H2O_started_from_R_laksh_fdd511 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   0.81 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    R Version:                  R version 3.6.1 (2019-07-05) 



In [54]:
set.seed(123)

In [55]:
N <- 1000

In [56]:
bloodTypes <- c('A', 'A', 'A', 'O', 'O', 'O','AB','B')

In [57]:
d <- data.frame(id = 1:N)

In [58]:
d$bloodType <- as.factor(bloodTypes[(d$id %% length(bloodTypes)) + 1])

In [59]:
d$age <- runif(N, min=18, max=65)


In [60]:
v = round(rnorm(N, mean =5, sd=2))
v =pmax(v,0)
v =pmin(v,9)
table(v)
d$healthyeating =v

v
  0   1   2   3   4   5   6   7   8   9 
  7  36  61 126 148 204 192 127  56  43 

In [61]:
v = round(rnorm(N, mean =5, sd = 2))
v = v+ifelse(d$age < 30, 1, 0)
v =pmax(v,0)
v =pmin(v,9)
table(v)
d$activelifecycle =v

v
  0   1   2   3   4   5   6   7   8   9 
  7  29  56  97 172 199 166 128  96  50 

In [62]:
v = 20000 + ((d$age * 3)^2)#base salary based on age
v = v+ (d$healthyeating * 500)
v = v -  (d$activelifecycle * 300)
v = v+runif(N, 0, 5000)
d$income = round(v, -2)#round to nearest 100

In [72]:
as.h2o(d, destination_frame = "people")

  |======================================================================| 100%


  id bloodType      age healthyeating activelifecycle income
1  1         A 31.51614             4               3  31300
2  2         A 55.05034             3               4  52500
3  3         O 37.22192             7               3  38700
4  4         O 59.50182             7               6  56800
5  5         O 62.20196             2               7  54900
6  6        AB 20.14116             5               9  23900

[1000 rows x 6 columns] 

In [73]:
people = h2o.getFrame("people")

In [74]:
summary(people)

Warning message in summary.H2OFrame(people):
"Approximated quantiles computed! If you are interested in exact quantiles, please pass the `exact_quantiles=TRUE` parameter."

 id               bloodType age             healthyeating   activelifecycle
 Min.   :   1.0   A :375    Min.   :18.02   Min.   :0.000   Min.   :0.000  
 1st Qu.: 250.8   O :375    1st Qu.:29.89   1st Qu.:4.000   1st Qu.:4.000  
 Median : 500.5   AB:125    Median :41.00   Median :5.000   Median :5.000  
 Mean   : 500.5   B :125    Mean   :41.37   Mean   :5.024   Mean   :5.225  
 3rd Qu.: 750.2             3rd Qu.:53.06   3rd Qu.:6.000   3rd Qu.:7.000  
 Max.   :1000.0             Max.   :64.97   Max.   :9.000   Max.   :9.000  
 income         
 Min.   :22600  
 1st Qu.:31378  
 Median :38980  
 Mean   :40520  
 3rd Qu.:48798  
 Max.   :64600  

In [75]:
parts <- h2o.splitFrame(people, c(0.8, 0.1),
                        destination_frames = c("ptrain","pvalid","ptest"),
                       seed = 123)

In [76]:
sapply(parts, nrow)

[1] 788 118  94

In [77]:
train <- parts[[1]]

In [78]:
train

  id bloodType      age healthyeating activelifecycle income
1  1         A 31.51614             4               3  31300
2  2         A 55.05034             3               4  52500
3  3         O 37.22192             7               3  38700
4  4         O 59.50182             7               6  56800
5  5         O 62.20196             2               7  54900
6  6        AB 20.14116             5               9  23900

[788 rows x 6 columns] 

In [79]:
valid <- parts[[2]]

In [80]:
test <-  parts[[3]]

In [81]:
train <- h2o.getFrame("ptrain")

In [82]:
train

  id bloodType      age healthyeating activelifecycle income
1  1         A 31.51614             4               3  31300
2  2         A 55.05034             3               4  52500
3  3         O 37.22192             7               3  38700
4  4         O 59.50182             7               6  56800
5  5         O 62.20196             2               7  54900
6  6        AB 20.14116             5               9  23900

[788 rows x 6 columns] 

In [84]:
valid <- h2o.getFrame("pvalid")

In [85]:
test <- h2o.getFrame("ptest")

In [86]:
y <- "income"

In [87]:
x <- setdiff(names(train), c("id", y))

In [88]:
x

[1] "bloodType"       "age"             "healthyeating"   "activelifecycle"

In [89]:
y

[1] "income"

In [90]:
m1 <- h2o.gbm(x, y, train, model_id = "defaults_r", validation_frame = valid)

  |======================================================================| 100%


In [91]:
h2o.performance(m1, train = TRUE)

H2ORegressionMetrics: gbm
** Reported on training data. **

MSE:  1153342
RMSE:  1073.938
MAE:  895.534
RMSLE:  0.02987908
Mean Residual Deviance :  1153342


In [92]:
h2o.performance(m1, valid = TRUE)

H2ORegressionMetrics: gbm
** Reported on validation data. **

MSE:  2821231
RMSE:  1679.652
MAE:  1444.85
RMSLE:  0.0452246
Mean Residual Deviance :  2821231


In [93]:
h2o.performance(m1, test)

H2ORegressionMetrics: gbm

MSE:  2525395
RMSE:  1589.149
MAE:  1365.796
RMSLE:  0.04294726
Mean Residual Deviance :  2525395


In [94]:
m2 <- h2o.gbm(x, y, train, model_id = "overfit_r", validation_frame = valid, ntrees = 1000, max_depth = 10)

  |======================================================================| 100%


In [95]:
h2o.performance(m1, train = TRUE)

H2ORegressionMetrics: gbm
** Reported on training data. **

MSE:  1153342
RMSE:  1073.938
MAE:  895.534
RMSLE:  0.02987908
Mean Residual Deviance :  1153342


In [96]:
h2o.performance(m1, valid = TRUE)

H2ORegressionMetrics: gbm
** Reported on validation data. **

MSE:  2821231
RMSE:  1679.652
MAE:  1444.85
RMSLE:  0.0452246
Mean Residual Deviance :  2821231


In [97]:
h2o.performance(m1, test)

H2ORegressionMetrics: gbm

MSE:  2525395
RMSE:  1589.149
MAE:  1365.796
RMSLE:  0.04294726
Mean Residual Deviance :  2525395
